<a href="https://colab.research.google.com/github/jesicagimenez94/DataAnalyst/blob/main/ETL_rickandmortyapi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import requests

# Extraemos los datos de la API de Rick and Morty
def extract_data_from_api():
    base_url = "https://rickandmortyapi.com/api/character"
    characters = []

    while base_url:
        response = requests.get(base_url)
        if response.status_code == 200:
            data = response.json()
            characters.extend(data.get("results", []))
            base_url = data.get("info", {}).get("next")  # Cambiar al siguiente paginado
        else:
            print(f"Error en la solicitud: {response.status_code}")
            break

    print(f"Se han extraído {len(characters)} personajes.")
    return characters

# Limpiamos y organizamos los datos
def transform_data(characters):
    df = pd.DataFrame(characters)

    # Dejamos solo las columnas importantes
    relevant_columns = ["id", "name", "status", "species", "type", "gender", "origin", "location"]
    df = df[relevant_columns]

    # Normalizamos las columnas de tipo diccionario (origin y location)
    df["origin_name"] = df["origin"].apply(lambda x: x["name"] if isinstance(x, dict) else "")
    df["location_name"] = df["location"].apply(lambda x: x["name"] if isinstance(x, dict) else "")
    df.drop(columns=["origin", "location"], inplace=True)

    print(f"Datos transformados, total de registros limpios: {df.shape[0]}")
    return df

# Cargamos el DataFrame en un archivo CSV
def load_data_to_csv(df, filename="rick_and_morty_characters.csv"):
    df.to_csv(filename, index=False)
    print(f"Datos cargados exitosamente en {filename}")

# Función que ejecuta el ETL
def run_etl():
    # Extracción
    characters = extract_data_from_api()

    # Transformación
    transformed_data = transform_data(characters)

    # Carga
    load_data_to_csv(transformed_data)

# Ejecutar el proceso ETL
run_etl()



Se han extraído 826 personajes.
Datos transformados, total de registros limpios: 826
Datos cargados exitosamente en rick_and_morty_characters.csv


In [ ]:
from google.colab import files
files.download("rick_and_morty_characters.csv")